Лабораторная работа 2. Обработка текста.

Задание:

Написать программу на языке программирования Python, реализующую:
1)	Предварительную обработку текста, включающую:
	-	Перевод текста в нижний регистр
	-	Удаление знаков препинания
	-	Удаление стоп-слов
	-	Удаление лишних символов
	-	Стемминг и лемматизацию
2)	Преобразование текста в мешок слов
3)	Преобразование текста в N-граммы, где N=2,4

Замечание!
Желательно провести предварительную обработку текста с использованием библиотеки NLTK.

In [258]:
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.snowball import SnowballStemmer # Стеммер для русского языка
from nltk.stem.porter import PorterStemmer # Стеммер для английского 
from nltk.stem import WordNetLemmatizer # Лемматайзер для английского
from nltk.corpus import wordnet
from pymystem3 import Mystem # Лемматайзер для русского языка 
import re
import string
from langdetect import detect
import logging
import os 
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from spellchecker import SpellChecker
from num2words import num2words
import requests


In [259]:
# Конфигурационный блок 

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\vadig\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [260]:
log_dir = r'D:\Projects\SvetlanaDmitrievna\lab3_ii\logs'
os.makedirs(log_dir, exist_ok=True) 
log_filename = os.path.join(log_dir, f'{datetime.now().strftime("%H %M %S %Y %m %d")} nlp.log')

# Удаляем логеры, которые уже запущены по случайности 
logger = logging.getLogger()
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
    
# Настройки для логгера 
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()  # Вывод в консоль 
    ]
)
logger = logging.getLogger(__name__)


In [ ]:
class TextPreprocessing: 
	"""
	Делает упорядоченную предобработку текста: 
	1. Перевод текста в нижний регистр. 
	2. Удаление пунтуации и других символов. 
	3. Удаление стоп-слов. 
	4. Нормализация текста. Лемматизация / стемминг. 
	"""

	def __init__(self, text, delete_punctuation_mode='string', text_normalization_method='lemma'):
		"""
		Инициализация класса TextPreprocessing.

		:param text: исходный текст для предобработки
		"""

		logger.info('Text is initializing...')

		self.text = text
		self.delete_punctuation_mode = delete_punctuation_mode
		self.language = self.__detect_language() 
		self.normalization_method = text_normalization_method

		logger.info(f'Text is initialized with parameters: delete_punctuation_mode={delete_punctuation_mode}, text_normalization_method={text_normalization_method}')
 
	def __detect_language(self) -> str: 
		"""
		Автоматическое определение языка с помощью библиотеки langdetect. 

		:return: Возвращается формат языка в стиле 'russian', 'english'.
		"""

		try:
			detected_lang = detect(self.text)
			
			lang_mapping = {
				'ru': 'russian',
				'en': 'english',
				# TODO: create more languages
			}

			language = lang_mapping[detected_lang]

			logger.info(f'Detected language: {language}')
			
			return language
		
		except:
			logger.critical('Error with detecting langauge.')

	def __lang_to_simple_form(self) -> str: 
		"""
		Вовзвращает упрощенную форму языка. 

		'english' -> 'en' 
		"""

		lang_mapping = {
				'russian': 'ru', 
				'english': 'en'
			}
		
		return lang_mapping[self.language]
 
	def __to_lower_case(self) -> str:
		"""
		Приводит текст к нижнему регистру. 

		:return: возвращаем измененный текст. 
		"""
		
		logger.info('Text has been converted to lower case.')

		return self.text.lower()
	
	def __correct_spell(self) -> str: 
		"""
		Исправляет опечатки в тексте. 
		
		:param text: исходный текст.
    :return: текст с исправленными опечатками.
		"""

		logger.info('Correction of typos initiated...')

		try:

			if self.language == 'english':
				spell = SpellChecker(language=self.__lang_to_simple_form()) 
				words = self.text.split() 
				corrected_words = [] 
				for word in words: 

					if word.isdigit() or len(word) <= 2: 
						corrected_words.append(word)
						continue

					corrected_word = spell.correction(word)
					corrected_words.append(corrected_word if corrected_word else word)

				logger.info(f'Correction of typos initiated done with {len(corrected_words)} corrected words.')
				corrected_text = ' '.join(corrected_words)

				return corrected_text
			
			# Для русского языка используем Яндекс.Спеллер
			elif self.language == 'russian':
					import requests
					
					url = "https://speller.yandex.net/services/spellservice.json/checkText"
					params = {
							'text': self.text,
							'lang': 'ru',
							'format': 'plain'
					}
					
					response = requests.get(url, params=params)
					
					if response.status_code == 200:
							corrections = response.json()
							result_text = self.text
							
							# Применяем исправления в обратном порядке (с конца текста)
							for item in reversed(corrections):
									pos = item['pos']
									length = item['len']
									suggestions = item['s']
									
									if suggestions:
											# Заменяем ошибочное слово на первое предложенное исправление
											result_text = result_text[:pos] + suggestions[0] + result_text[pos + length:]
							
							logger.info(f'Corrected {len(corrections)} typos using Yandex.Speller')
							return result_text
					else:
						logger.error(f'Yandex.Speller API returned status code {response.status_code}')
						return self.text

		except Exception as e: 
			logger.error(f'Typos correction error: {str(e)}')
			return self.text
	
	def __numbers_to_words(self) -> str: 
		"""
    Заменяет числа в тексте на их текстовые эквиваленты.
    
    Например:
    - "10 книг" -> "десять книг"
    - "2 students" -> "two students"
    
    :return: текст с числами, преобразованными в слова
    """
		
		logger.info('Converting numbers to words...')
		try: 

			num_lang = self.__lang_to_simple_form()

			def replace_number(match): 
				'''
				Вызывается, когда находится соответствие маске '\b\d+[,.]?\d*\b'

				:param match: это часть текста, которая маске подошла.
				'''
				try: 
					# .group(0) - возвращает весь текст, а не только его какую-то группу. 
					number = match.group(0)

					if '.' in number or ',' in number:
						number = number.replace(',', '.')
						return num2words(float(number), lang=num_lang)
					else: 
						return num2words(int(number), lang=num_lang)
					
				except ValueError: 
					
					return match.group(0)
					
			text_with_words = re.sub(r'\b\d+[,.]?\d*\b', replace_number, self.text)

			logger.info('Numbers have been converted to words.')
			return text_with_words 
		
		except Exception as e: 
			logger.error(f'Error during number to word conversion: {str(e)}')
			return self.text

	def __delete_punctuation_marks(self) -> str:
		"""
		Удаляет знаки препинания из предложения.

		Режимы (delete_punctuation_mode): 
		1. 're' — c помощью регулярных выражений. 
		2. 'string' — с помощью библиотеки string.  

		:return: текст без знаков препинания. 
		"""

		logger.info('Punctuation are being deleting...')

		match self.delete_punctuation_mode: 
			case 're':

				logger.info('Punctuation has been deleted.')

				return re.sub(r'[^\w\s]', '', self.text)
			
			case 'string':
				# Параметры: 
				# Первый аргумент '' символы для замены 
				# Второй аргумент '' символы на которые заменяем 
				# Третий аргумент string.punctuation символы, которые нужно удалить 
				# string.punctuation = "!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~" 
				translator = str.maketrans('', '', ''.join(c for c in string.punctuation if c != ' '))

				logger.info('Punctuation has been deleted.')

				# translate() применяет таблицу преобразования к строке 
				return self.text.translate(translator)

	def __delete_stop_words(self) -> str: 
		"""
		Удаляет стоп-слова. 

		Стоп-слова — распространенные слова в языке, которые не несут значимой смысловой нагрузки 
		при анализе текста. 

		Зачем их удалять? 
		1. Текст становится меньше. 
		2. Фокус на важных словах. 
		"""

		logger.info('Stop words are being removed...')

		stopwords.words(self.language)

		stop_words = set(stopwords.words(self.language))
		
		words = self.text.split()

		filtered_words = [word for word in words if word not in stop_words]

		logger.info('Stop words have been removed.')
		
		return ' '.join(filtered_words)
		
	def __stemming(self) -> str: 
		"""
		Выполняет стемминг слов в тексте.
		
		Стемминг — процесс нахождения основы слова (стеммы) путем 
		отбрасывания окончаний и суффиксов. Например, для слов "рыбак", 
		"рыбачить", "рыбный" стеммой будет "рыб".
		
		:return: текст со словами, приведенными к их основам. 
		"""

		logger.info('Stemming is being performed...')
		
		try: 
			
			words = self.text.split() 
			stemmed_words = [] 

			match self.language: 
				case 'russian': 
					stemmer = SnowballStemmer("russian")
				case 'english': 
					stemmer = PorterStemmer()
				case _: 
					logger.error(f'Stemming not supported for language: {self.language}')
					return self.text

			for word in words:
				stemmed_words.append(stemmer.stem(word))
			
			logger.info('Stemming has been performed.')
			return ' '.join(stemmed_words)
			
		except Exception as e: 
			logger.error(f'Error during stemming: {str(e)}')
			return self.text
		
	def __lemming(self) -> str: 
		"""
		Выполняет лемматизацию слов в тексте.
			
		Лемматизация — процесс приведения слова к его словарной форме (лемме).
		Например, для слов "бегущий", "бегает", "бежит" леммой будет "бежать".
			
		:return: текст со словами в словарной форме
		"""

		logger.info('Lemmatizaion is being performed...')

		try: 
			
			words = self.text.split() 
			lemmatized_words = []

			match self.language: 
				case 'russian': 
						lemmatizer = Mystem()

						for word in words: 
							try: 
								# .parse(word) — библиотека pymorphy2 анализирует слово и 
								# возвращает список всех возможных морфологических разборов этого слова.
								# мы берем [0] — наиболее вероятный
								#
								# После чего делаем нормальную форму (лемму) .normal_form
								# Для существительных - именительный падеж, единственное число
								# Для глаголов - инфинитив
								# Для прилагательных - мужской род, единственное число, именительный падеж
								lemma = ''.join(lemmatizer.lemmatize(word)).strip()
								lemmatized_words.append(lemma)
							
							except Exception as e:
								logger.warning(f"Failed to lemmatize word '{word}': {e}")
								lemmatized_words.append(word)  # Сохраняем исходное слово

				case 'english':

					lemmatizer = WordNetLemmatizer()

					# Разобраться, что делает эта функция 
					def get_wordnet_pos(word):
						"""Отображает тег части речи NLTK на тег WordNet"""
						tag = nltk.pos_tag([word])[0][1][0].upper()
						tag_dict = {
								'J': wordnet.ADJ,
								'N': wordnet.NOUN,
								'V': wordnet.VERB,
								'R': wordnet.ADV
						}
						return tag_dict.get(tag, wordnet.NOUN)
					
					for word in words: 
						lemma = lemmatizer.lemmatize(word, get_wordnet_pos(word))
						lemmatized_words.append(lemma)

				case _: 
					logger.error(f'Lemmatization not supported for language: {self.language}')
					return self.text
			
			logger.info('Lemmatization has been performed.')
			return ' '.join(lemmatized_words)
			
		except Exception as e: 
			logger.error(f'Error during lemmatizaion: {str(e)}')
			return self.text

	def __text_normalization(self) -> str: 
		"""
		Выбирает метод нормализации текста. 
		"""

		match self.normalization_method: 
			case 'lemma': 
				return self.__lemming()
			case 'stem': 
				return self.__stemming()
			case _: 
				logger.warning('There is no such thing as a text normalization method')
				logger.info('Returned text without normalization')
				return self.text

	def process_text(self) -> str:
		"""
		Обрабатывает текст, применяя все методы предобработки.
		
		:return: обработанный текст по всем функциям.
		""" 

		for func in [self.__to_lower_case, self.__numbers_to_words, self.__delete_punctuation_marks, self.__correct_spell, self.__delete_stop_words, self.__text_normalization]: 
				self.text = func()
		
		logger.info('Text preprocessing is done!')

		return self.text 

<>:160: SyntaxWarning: invalid escape sequence '\d'
<>:160: SyntaxWarning: invalid escape sequence '\d'
C:\Users\vadig\AppData\Local\Temp\ipykernel_3048\56961097.py:160: SyntaxWarning: invalid escape sequence '\d'
  '''


In [262]:
class BagOfWords: 
	"""
	Преобразует текст в мешок слов. 

	Мешок слов (Bag of Words) - это представление текста, которое описывает 
  частоту появления слов в документе. Это простая модель, игнорирующая 
  порядок слов, но сохраняющая их количество.  
	"""
	
	def __init__(self, preprocessed_text): 
		"""
		Инициализация класса. 

		:param preprocessed_text: предобработанный текст 
		"""

		logger.info('BagOfWords class is initialized')

		if isinstance(preprocessed_text, dict):
				logger.warning('Input is already a dictionary, not a text string')
				self.preprocessed_text = ' '.join(preprocessed_text.keys())
		else:
				self.preprocessed_text = preprocessed_text
	
	def create_bag(self): 
		"""
    Создает мешок слов с помощью sklearn.
    
    :return: словарь, где ключи - слова, значения - частота их появления
    """
		
		logger.info('Creating bag of words using sklearn...')
		
		try: 
			vectorizer = CountVectorizer()

			# fit() — обучает векторизатор 
			# transform() — преобразует текст в числовое представление
			# преобразует текст в матрицу: 
			# 1. строки соответствуют документа (в моем случае один)
			# 2. столбцы соответствуют уникальным словам из словаря 
			# 3. значения в ячейках — частота появления слов в соответствующем документе 
			#
			# X — объект scipy.sparse.csr_matrix (разреженная матрица)
			# CSR — compressed sparse row 
			# такая матрица используется для эффективного хранения данных, где большинство элементов 
			# равны нулю. 
			#
			# Пример матрицы: 
			# (0, 1)	1
			# (0, 3)	2
			# (0, 4)	1
			# (0, 7)	1
			# (0, 10)	1
			#
			# (строка, столбец) сколько раз встречается 
			X = vectorizer.fit_transform([self.preprocessed_text])
			words = vectorizer.get_feature_names_out()

			print(words)

			word_freq = {} 
			for i, word in enumerate(words): 
				count = X.toarray()[0][i]
				if count > 0: 
					word_freq[word] = count 

			logger.info(f'Bag of words created with {len(word_freq)} unique words using sklearn')
			return word_freq 
		
		except Exception as e: 
			logger.info(f'Bag of words created with {len(word_freq)} unique words using sklearn')
			return {} 

In [263]:
class NGrams: 
	"""
	Преобразует текст в N-граммы - последовательности из N слов.
	
	N-грамма - это последовательность из N элементов (в данном случае, слов)
	идущих друг за другом в тексте.
	"""

	def __init__(self, preprocessed_text):
		"""
		Инициализация класса.
		
		:param preprocessed_text: предобработанный текст
		"""

		logger.info('NGrams class is initialized')

		self.preprocessed_text = preprocessed_text

	def create_ngrams(self, n=2):
		"""
		Создаёт N-граммы из предобработанного текста. 

		:param n: размер N-граммы (количество слов в одной N-грамме)
    :return: словарь, где ключи - N-граммы, значения - частота их появления
		"""

		logger.info(f'Creating {n}-grams...')

		try: 
			words = self.preprocessed_text.split() 

			# Если текст короче, чем размер N-граммы, возвращаем пустой словарь
			if len(words) < n:
					logger.warning(f'Text is too short to create {n}-grams')
					return {}
      
			ngrams = []
			for i in range(len(words) - n + 1): 
				ngram = ' '.join(words[i:i+n])
				ngrams.append(ngram)

			ngram_freq = {} 
			for ngram in ngrams: 
				if ngram in ngram_freq: 
					ngram_freq[ngram] += 1 
				else: 
					ngram_freq[ngram] = 1 
			
			logger.info(f'{n}-grams created with {len(ngram_freq)} unique {n}-grams')
			return ngram_freq 
	
		except Exception as e: 
			logger.error(f'Error during {n}-grams creation: {str(e)}')
			return {} 


In [264]:
# Для работы с txt 

def read_txt(file_path): 
	"""
	Считывает текст из файла. 

	:param file_path: путь к файлу 
	:return: содержимое файла в виде строки 
	"""

	logger.info(f'Reading text from file: {file_path}')

	try: 
		with open(file_path, 'r', encoding='utf-8') as file: 
			text = file.read()

		logger.info(f'Successfully read {len(text)} chars from file.')

		return text
	
	except Exception as e: 
		logger.critical(f'Didn`t get a chance to read the text: {str(e)}')
		return ""
	
	
def text_to_txt(file_path, text): 
	"""
	Записывает текст в txt. 

	:param file_path: путь к файлу, куда записать текст
	:param text: сам обработанный текст
	"""

	logger.info(f'Transmitted preprocessed text with {len(text)} chars.')

	try: 
		with open(file_path, 'w', encoding='utf-8') as file: 
			
			# Этот блок кода сделан специально, чтобы текст можно было читать не одним
			# сплошным предложением
			text = text.split()
			line = ''
			result = []
			for i, word in enumerate(text): 
				if i % 15 == 0 and i != 0: 
					line.rstrip()
					line += '\n'
					result.append(line)
					line = f'{word} '
				else: 
					line += f'{word} '

			if line: result.append(line)

			result = ''.join(result)

			file.write(result)

			logger.info(f'Text successfully was written to {file_path}')

	except Exception as e: 
		logger.critical(f'Error to write the text: {str(e)}')

In [265]:
russian_input_file_path = 'D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/test_text.txt'
text = read_txt(russian_input_file_path)

output = TextPreprocessing(
	text, 
	delete_punctuation_mode='string', 
	text_normalization_method='lemma').process_text()

2025-03-26 21:57:24,335 - INFO - Reading text from file: D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/test_text.txt
2025-03-26 21:57:24,336 - INFO - Successfully read 190 chars from file.
2025-03-26 21:57:24,337 - INFO - Text is initializing...
2025-03-26 21:57:24,340 - INFO - Detected language: russian
2025-03-26 21:57:24,341 - INFO - Text is initialized with parameters: delete_punctuation_mode=string, text_normalization_method=lemma
2025-03-26 21:57:24,343 - INFO - Text has been converted to lower case.
2025-03-26 21:57:24,344 - INFO - Converting numbers to words...
2025-03-26 21:57:24,346 - INFO - Numbers have been converted to words.
2025-03-26 21:57:24,347 - INFO - Punctuation are being deleting...
2025-03-26 21:57:24,349 - INFO - Punctuation has been deleted.
2025-03-26 21:57:24,350 - INFO - Correction of typos initiated...
2025-03-26 21:57:25,382 - INFO - Corrected 0 typos using Yandex.Speller
2025-03-26 21:57:25,382 - INFO - Text preprocessing is done!


In [266]:
# Записываем текст в файл 

russian_output_file_path = 'D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/output_russian.txt'
text_to_txt(russian_output_file_path, output)

2025-03-26 21:57:25,390 - INFO - Transmitted preprocessed text with 254 chars.
2025-03-26 21:57:25,391 - INFO - Text successfully was written to D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/output_russian.txt


In [267]:
bag_of_words = BagOfWords(output).create_bag()
bigrams = NGrams(output).create_ngrams(n=2)
ngrams = NGrams(output).create_ngrams(n=4)


print(bag_of_words)
print(bigrams)
print(ngrams)

2025-03-26 21:57:25,399 - INFO - BagOfWords class is initialized
2025-03-26 21:57:25,400 - INFO - Creating bag of words using sklearn...
2025-03-26 21:57:25,404 - INFO - Bag of words created with 32 unique words using sklearn
2025-03-26 21:57:25,405 - INFO - NGrams class is initialized
2025-03-26 21:57:25,405 - INFO - Creating 2-grams...
2025-03-26 21:57:25,406 - INFO - 2-grams created with 36 unique 2-grams
2025-03-26 21:57:25,407 - INFO - NGrams class is initialized
2025-03-26 21:57:25,408 - INFO - Creating 4-grams...
2025-03-26 21:57:25,408 - INFO - 4-grams created with 34 unique 4-grams


['бизнеса' 'будет' 'вторая' 'дeла' 'два' 'двадцать' 'жизнь' 'интересных'
 'как' 'надеюсь' 'одиннадцать' 'пoвседневную' 'пoговорить' 'первая'
 'полезно' 'привет' 'пятьсот' 'секрета' 'семь' 'сотых' 'способов' 'сто'
 'темах' 'три' 'тридцать' 'тысячных' 'улучшить' 'успешного' 'хoчу' 'целых'
 'четыре' 'это']
{'бизнеса': 1, 'будет': 1, 'вторая': 1, 'дeла': 1, 'два': 1, 'двадцать': 1, 'жизнь': 1, 'интересных': 1, 'как': 1, 'надеюсь': 1, 'одиннадцать': 1, 'пoвседневную': 1, 'пoговорить': 1, 'первая': 1, 'полезно': 1, 'привет': 1, 'пятьсот': 1, 'секрета': 1, 'семь': 1, 'сотых': 1, 'способов': 1, 'сто': 1, 'темах': 1, 'три': 2, 'тридцать': 1, 'тысячных': 1, 'улучшить': 1, 'успешного': 1, 'хoчу': 1, 'целых': 2, 'четыре': 1, 'это': 2}
{'привет как': 1, 'как дeла': 1, 'дeла я': 1, 'я хoчу': 1, 'хoчу пoговорить': 1, 'пoговорить о': 1, 'о два': 1, 'два интересных': 1, 'интересных темах': 1, 'темах первая': 1, 'первая это': 1, 'это семь': 1, 'семь способов': 1, 'способов улучшить': 1, 'улучшить пoвсед

In [268]:
english_input_file_path = 'D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/english_text.txt'
text = read_txt(english_input_file_path)

eng_output = TextPreprocessing(
	text, 
	text_normalization_method='lemma', 
	delete_punctuation_mode='string').process_text()

2025-03-26 21:57:25,418 - INFO - Reading text from file: D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/english_text.txt
2025-03-26 21:57:25,419 - INFO - Successfully read 850 chars from file.
2025-03-26 21:57:25,419 - INFO - Text is initializing...
2025-03-26 21:57:25,424 - INFO - Detected language: english
2025-03-26 21:57:25,425 - INFO - Text is initialized with parameters: delete_punctuation_mode=string, text_normalization_method=lemma
2025-03-26 21:57:25,425 - INFO - Text has been converted to lower case.
2025-03-26 21:57:25,426 - INFO - Converting numbers to words...
2025-03-26 21:57:25,427 - INFO - Numbers have been converted to words.
2025-03-26 21:57:25,427 - INFO - Punctuation are being deleting...
2025-03-26 21:57:25,428 - INFO - Punctuation has been deleted.
2025-03-26 21:57:25,428 - INFO - Correction of typos initiated...
2025-03-26 21:57:26,014 - INFO - Correction of typos initiated done with 134 corrected words.
2025-03-26 21:57:26,018 - INFO - Text preprocessing is done!


In [269]:
# Записываем текст в файл 

english_output_file_path = 'D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/output_english.txt'
text_to_txt(english_input_file_path, eng_output)

2025-03-26 21:57:26,024 - INFO - Transmitted preprocessed text with 841 chars.
2025-03-26 21:57:26,026 - INFO - Text successfully was written to D:/Projects/SvetlanaDmitrievna/lab3_ii/texts/english_text.txt


In [270]:
bag_of_words = BagOfWords(eng_output).create_bag()
bigrams = NGrams(eng_output).create_ngrams(n=2)
ngrams = NGrams(eng_output).create_ngrams(n=4)

print(bag_of_words)
print(bigrams)
print(ngrams)

2025-03-26 21:57:26,034 - INFO - BagOfWords class is initialized
2025-03-26 21:57:26,035 - INFO - Creating bag of words using sklearn...
2025-03-26 21:57:26,037 - INFO - Bag of words created with 119 unique words using sklearn
2025-03-26 21:57:26,038 - INFO - NGrams class is initialized
2025-03-26 21:57:26,038 - INFO - Creating 2-grams...
2025-03-26 21:57:26,039 - INFO - 2-grams created with 133 unique 2-grams
2025-03-26 21:57:26,039 - INFO - NGrams class is initialized
2025-03-26 21:57:26,040 - INFO - Creating 4-grams...
2025-03-26 21:57:26,041 - INFO - 4-grams created with 131 unique 4-grams


['able' 'although' 'arch' 'arm' 'armourlike' 'back' 'bed' 'bedding'
 'belly' 'bit' 'boa' 'brown' 'collection' 'compare' 'could' 'cover' 'cut'
 'divide' 'domed' 'dream' 'drop' 'dull' 'familiar' 'feel' 'fit' 'forget'
 'found' 'four' 'frame' 'fur' 'get' 'gild' 'gregory' 'happen' 'hardly'
 'hat' 'head' 'heard' 'heavy' 'helplessly' 'hit' 'horrible' 'house'
 'however' 'human' 'hung' 'illustrate' 'lady' 'lay' 'leg' 'lift' 'little'
 'longer' 'look' 'low' 'magazine' 'make' 'many' 'moment' 'morning' 'muff'
 'nice' 'nonsense' 'one' 'pane' 'peacefully' 'picture' 'pitifully'
 'position' 'present' 'proper' 'quite' 'rain' 'raise' 'ready' 'recently'
 'rest' 'right' 'room' 'sad' 'salesman' 'salsa' 'sample' 'sat' 'section'
 'see' 'seem' 'show' 'size' 'sleep' 'slide' 'slightly' 'small' 'something'
 'spread' 'state' 'stiff' 'table' 'textile' 'thin' 'thought' 'towards'
 'transform' 'travel' 'troubled' 'turn' 'unable' 'upright' 'use' 'vermin'
 'viewer' 'wall' 'want' 'wave' 'weather' 'whats' 'whole' 'window'